<a href="https://colab.research.google.com/github/hwankang/KoChatGPT/blob/main/gpt2_june_19_17pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==2.11.0
!pip install tensorflow==2.2.0
!pip install sentencepiece==0.1.85
!pip install gluonnlp==0.9.1
!pip install mxnet==1.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.8/674.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 71.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=0935cb179

In [2]:
!mkdir -p gpt2
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt \
    -O gpt2/finetune_data.txt

--2023-06-19 05:38:15--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24570 (24K) [text/plain]
Saving to: ‘gpt2/finetune_data.txt’

gpt2/finetune_data. 100%[===================>]  23.99K  --.-KB/s    in 0.001s  

2023-06-19 05:38:15 (15.7 MB/s) - ‘gpt2/finetune_data.txt’ saved [24570/24570]



In [6]:
import os
import numpy as np
import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from nltk.tokenize import sent_tokenize
!pip install transformers
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import TFGPT2LMHeadModel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip  -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

--2023-06-19 05:58:20--  https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip [following]
--2023-06-19 05:58:20--  https://www.dropbox.com/s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc968c2817f6e4e7b31fb71cb2cc.dl.dropboxusercontent.com/cd/0/inline/B-Q2h3tjpIZBL-Xv-vKZCAnJ3Fmle48IVMyVkBqOd-SGhTSbAUsEOqG7zM7KtLueiwvBVmjFeiWdu8VH0aB31I4ZH_A3aeJQrEnNp4cYnIE575cMT-I01z5AjKQl_2k4T2rBTI-x0vXo_ohPLgMMfd5ccC850qhRhBH-njoy2wojdQ/file# [following]
--2023-06-19 05:58:21--  https://uc968c2817f6e4e7b31fb71cb2cc.dl.dropboxusercontent.com/cd/0/inline/B-Q2h3tjpIZBL-Xv-vKZCAnJ3Fmle48IVMyVkBqOd-SGhTSbAUsEOqG7zM7KtLueiwvBVmjFeiWdu8VH

In [26]:
class GPT2Model(tf.keras.Model):
    def __init__(self,dir_path):
        super(GPT2Model,self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)

    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [32]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [41]:
#!pip install Sentencepiece==0.1.85
!pip install Sentencepiece
BATCH_SIZE =16
NUM_EPOCHS =10
MAX_LEN = 30
TOKENIZER_PATH='./gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab=nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                            mask_token=None,
                                            sep_token=None,
                                            cls_token=None,
                                            unknown_token='<unk>',
                                            padding_token='<pad>',
                                            bos_token='<s>',
                                             eos_token='</s>')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


In [89]:
import tensorflow as tf
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k >0:
        indices_to_remove = logits < tf.math.top_k(logits,top_k)[0][...,-1,None]
        _logits[indices_to_remove]=filter_value

    if top_p >0.0:
        sorted_logits=tf.sort(logits, direction='DESCENDING')
        sorted_indices=tf.argsort(logits, direction='DESCENDING')
        cumulative_probs=tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1),axis=-1)

        sorted_indices_to_remove =cumulative_probs >top_p
        sorted_indices_to_remove =tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove=sorted_indices[sorted_indices_to_remove].numpy().tolist()

        _logits[indices_to_remove]=filter_value
    return tf.constant([_logits])

In [90]:
def generate_sentence(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sentence = seed_word
    token =tokenizer(sentence)

    for _ in range(max_step):
        input_ids=tf.constant([vocab[vocab.bos_token],]+vocab[token])[None,:]
        outputs=model(input_ids)[:,-1,:]
        if greedy:
            gen=vocab.to_tokens(tf.argmax(outputs,axis=-1).numpy().tolist()[0])
        else:
            output_logit =tf_top_k_top_p_filtering(outputs[0],top_k=top_k, top_p=top_p)
            gen=vocab.to_tokens(tf.random.categorical(output_logit,1).numpy().tolist()[0])[0]
        if gen =='</s>':
            break
        sentence +=gen.replace('_',' ')
        token = tokenizer(sentence)
    return sentence


In [92]:
generate_sentence('오늘', gpt_model, greedy=True)
#generate_sentence('오늘', gpt_model, top_k=0, top_p=0.95)

RuntimeError: ignored

In [72]:
import tensorflow as tf

def generate_sentence(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sentence = seed_word
    token = tokenizer(sentence)

    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]+vocab[token])[None,:]
        outputs = model(input_ids)[:,-1,:]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs,axis=-1).numpy().tolist()[0])
        else:
            output_logit =tf_top_k_top_p_filtering(outputs[0],top_k=top_k, top_p=top_p)
            gen=vocab.to_tokens(tf.random.categorical(output_logit,1).numpy().tolist()[0])[0]
        if gen =='</s>':
            break
        sentence +=gen.replace('-',' ')
        token = tokenizer(sentence)
    return sentence

model = tf.keras.models.load_model('model.h5')
vocab = tf.keras.datasets.imdb.get_word_index()

sentence = generate_sentence('The cat', model, max_step=100, greedy=True)

print(sentence)

OSError: ignored

https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip